Rename a bunch of files using Git and renamer?



In [ ]:
var renamer = require('renamer');

function renameUsingGit(gitRoot, match, find, replace) {
    var files = renamer.expand(path.join(gitRoot, match));
    var results = renamer.replace({
        files: files.filesAndDirs,
        find: find,
        replace: replace
    });
    return renamer.dryRun(results).list.map(r => {
        // rename with git instead
        var output = execSync('git '
            + '--work-tree=' + JSON.stringify(gitRoot)
            + ' --git-dir=' + JSON.stringify(path.join(gitRoot, '.git'))
            + ' mv ' + JSON.stringify(r.before)
            + ' ' + JSON.stringify(r.after));
        return output.toString() || r.after;
    });
};
renameUsingGit;


In [ ]:
// test it
// rename all scss files to less
//renameUsingGit(
//    '/Users/briancullinan/Documents/studysauce4',
//    'src/**/*.css',
//    '.css',
//    '.scss');


json git tree?


In [ ]:
var execSync = require('child_process').execSync;
/*var {
    execCmd
} = importer.import([
    'spawn child process',
]);*/


function gitTree(project) {
    console.log('creating tree "' + project + '"');
    var branches = execSync('git branch --list --all ', {cwd: project}).toString()
        .trim().split(/\s*\n\s*\**\s*/ig)
        .map(b => b.replace('remotes/', '').trim());

    // reverse git so we can add merges to the "graph" after the branch tree is constructed
    var branch = execSync('git log --pretty=format:"%ct %d %h %p" '
        + '--reverse --branches --tags ', {cwd: project}).toString().split(/\s*\n\s*/ig);
    var all = []; //execSync('git log --pretty=format:"%h %p" --reverse --all').toString().split(/\s*\n\s*/ig);

    var nodes = [];
    var edges = [];
    branch.forEach(b => {
        var commits = ((/[0-9]+\s+\(.*?\)\s*(.+)/ig).exec(b)
            || (/[0-9]+\s+(.+)/ig).exec(b) || [b, b.trim()])[1].trim().split(/\s+/ig);
        commits.forEach(r => {
            if (nodes.filter(d => d.name == r).length == 0) {
                nodes[nodes.length] = {
                    name: r,
                    index: nodes.length,
                    branch: ((/[0-9]+\s+\((.*?)\)/ig).exec(b) || [b, ''])[1],
                    time: (/([0-9]+)\s+/ig).exec(b)[1]
                }
            }
        });
        // don't include merges in tree
        if (commits.length == 3) {
            commits = [commits[0], commits[1]];
        }
        if (commits.length == 2) {
            edges = edges.concat(commits.slice(1)
                // check all existing sources to test for merge
                .map((c, i) => {
                    return {source: c, target: commits[i]}; // -1 because of slice(1)
                }))
        }
    });

    return formatNodes(nodes, edges.reverse());
}
module.exports = gitTree;



tip git tree?

just the tip?

source tree?



In [ ]:
var importer = require('../Core');
var _ = require('underscore');
var fs = require('fs');
var path = require('path');
var {
    formatNodes, gitTree, displayBranches,
    cloneProject, execCmd
} = importer.import([
    'spawn child process',
    'd3.ipynb[format d3 tree]',
    'json git tree',
    'd3.ipynb[display d3 tree]',
    'git project directory'
]);

// TODO: convert this to bash
function getResetCommit(branch, parent, project) {
    branch = branch.replace('HEAD -> ', '').trim();
    parent = parent.replace('HEAD -> ', '').trim();
    return execCmd(`
git checkout ${branch}
git reset --mixed ${parent}
git add -A
git commit -m "auto reset ${branch}"
`, {cwd: project});
}

var svgs = [];
function resetAllBranches() {
    var children = [];
    var nodeTree = _.sortBy(gitTree(project), a => a.time);
    var promises = [];
    for (const n of nodeTree) {
        var current = n;
        var parentCount = 0;
        while (typeof current.parent !== 'undefined'
            && current.parent.branch === current.branch) {
            current = current.parent;
            parentCount++;
        }
        if (parentCount > 0 && typeof current.parent !== 'undefined') {
            // TODO: keep track of the parent so the squashed branch can be merged on to the newly squashed parent
            promises.push(((current, n) => resolve => {
                if (typeof n.children !== 'undefined') {
                    children.push(n);
                }
                return getResetCommit(n.branch, current.parent.name, project)
                    .then(() => {
                        var nodeTree = _.sortBy(gitTree(project), a => a.time);
                        svgs.push(displayBranches(nodeTree));
                        resolve();
                    })
            })(current, n));
        }
    }
    return importer.runAllPromises(promises)
        .then(() => children);
}

function gitTipOfTree(project) {
    svgs = [];
    return cloneProject(project, true)
        // TODO: git checkout command
        // TODO: pull all missing remmotes
        .then(r => {
            project = r;
            var nodeTree = _.sortBy(gitTree(project), a => a.time);
            const head = nodeTree.filter(r => r.branch.indexOf('HEAD') > -1)[0];
            const branch = head
                ? (nodeTree.filter(r => r.branch.indexOf(head.branch.replace(/HEAD *-*>* */ig, '').split(/\s*,\s*/ig)) > -1)[0] || nodeTree[0])
                : nodeTree[0];
            return displayBranches([branch]);
        })
        .then(r => svgs.push(r))
        //.then(() => resetAllBranches())
        .then(() => svgs)
};
module.exports = gitTipOfTree;


In [ ]:
var importer = require('../Core');
var gitTipOfTree = importer.import('git.ipynb[tip git tree]');

$$.async();
gitTipOfTree('kotlin')
    .then(r => $$.html(r.join('')))
    .catch(e => $$.sendError(e))


Display graphs of different commands and scenarios using dry-run?

https://github.com/kpdecker/jsdiff

http://incaseofstairs.com/jsdiff/



In [ ]:
var path = require('path');
var gulp = require('gulp');
var run = require('gulp-run');
var tap = require('gulp-tap');
var importer = require('../Core');
var gulpPromise = importer.import('gulp task to a Promise', {gulp})

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/portal') : project;

function getBranchBoundaries(p) {
    if(p) {
        project = p;
    }

    gulp.task('git watch', function () {
        return run('git diff -w a229417..498d5a5', {cwd: project})
            .pipe(tap(file => console.log(file)))
            .pipe(gulp.dest('output'))
    });

    return gulpPromise(['git watch'])
}
module.exports = getBranchBoundaries;

if(typeof $$ !== 'undefined') {
    $$.async();
    getBranchBoundaries(project)
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}



update git?


In [ ]:
var importer = require('../Core');
var createNewEvent = importer.import('create new calendar event');

function resetRebaseInstallEvent(project) {
    if(!project) {
        project = path.resolve(__dirname, '..', '..', 'jupytangular');
    }
    return createNewEvent('spawn child process', JSON.stringify({
        script: importer.interpret('git auto rebase').code + '\nnpm install',
        options: {cwd: project} // TODO: fix current working directory using project name?
    }, null, 4), {calendarId: 'aws'})
}
module.exports = resetRebaseInstallEvent;


github updates?

check github for commits since the current one?



In [1]:
var _ = require('lodash');
var {JSDOM} = require('jsdom');
var importer = require('../Core');
if(typeof importer === 'undefined') {
    throw new Error('this should never happen');
}
var {
    gitTree, execCmd, getOauthClient, createNewEvent,
    updateEvent, cloneProject, getDaysEvents
} = importer.import([
    'd3.ipynb[format d3 tree]',
    'json git tree',
    'spawn child process',
    'import google calendar api',
    'create merge delete event',
    'git project directory',
    'days events',
    'create new calendar event'
]);

var options = {
    calendarId: 'aws'
}

// create a calendar event to retest this branch
function installBuildTestEvent(project) {
    return createNewEvent('spawn child process', JSON.stringify({
        script: `npm install\nnpm run build\nnpm run test`,
        options: {cwd: project} // TODO: fix current working directory using project name?
    }, null, 4), options)
}

function updateHeartbeatEvent(project, branch, commit) {
    return getOauthClient(options)
        .then(() => getDaysEvents(new Date(), options))
        .then(r => {
            const heartbeat = r.filter(r => r.event.summary === 'heartbeat'
                                       || r.event.summary === 'todays heartbeat items');
            try {
                var dom = new JSDOM('<body>' + (heartbeat[0].event.description || '').replace(/<br\/?>/igm, '\n') + '</body>');
                const desc = dom.window.document.body.textContent.split('\n');
                desc.forEach((c, i) => {
                    if(c.trim().length > 0 && c.includes('github updates')) {
                        const parameters = JSON.parse(((/[\{].*[\}]|[\[].*[\]]|(['"]).*\1/igm).exec(c) || ['""'])[0]);
                        if(parameters[0] === project && parameters[1] === branch) {
                            parameters[2] = commit;
                        }
                        desc[i] = 'github updates ' + JSON.stringify(parameters);
                    }
                });
                assert(desc.length, 'there is a serious problem with updating the commit');
                heartbeat[0].event.description = desc.join('\n');
                return updateEvent(heartbeat[0].event, options);
            } catch ( e ) {
                console.log(e);
            }
        })
}

function findBranchRemote(project, branch) {
    return execCmd(`git branch -r`, {cwd: project}) // 
        .then(r => {
            const remotes = r.join('\n').split(/\s*\n\s*/igm).filter(r => (r.split('/')[1] || '').trim() === branch);
            if(remotes.length === 0) {
                throw new Error('branch "' + branch + '" not found on any remotes ' + JSON.stringify(r))
            }
            remote = remotes[0].split('/')[0];
            return remote;
        })
        .then(() => ({remote, branch}))
}

function getUpdate(project, branch, commit) {
    var projectCopy;
    return cloneProject(project)
        .then(p => {
            projectCopy = p;
            return findBranchRemote(projectCopy, branch);
        })
        .then(({remote, branch}) => execCmd(`
git fetch ${remote}
git log ${commit || 'HEAD'}..${remote}/${branch}`, {cwd: projectCopy})
            .then(log => {
                if(log.join('').trim().length > 0) {
                    const commit = (/commit (.*)/ig).exec(log.join(''))[1];
                    return cloneProject(projectCopy, true)
                        .then(r => {
                            projectCopy = r;
                            if(branch) {
                                return execCmd(`
git reset --hard
git checkout ${branch}`, {cwd: projectCopy})
                            }
                        })
                        .then(() => execCmd(importer.interpret('git auto rebase').code.trim()
                                            + ` ${remote} ${branch}`, {cwd: projectCopy}))
                        .then(() => installBuildTestEvent(projectCopy))
                    // TODO: update heartbeat event description to pass in the latest commit
                        .then(() => updateHeartbeatEvent(project, branch, commit))
                } else {
                    throw new Error('nothing for git to do');
                }
            }))
}
module.exports = getUpdate;

// git log currentbranch@{1}..currentbranch
// where the @{1} notation means "the commit the branch pointed to just before it last got updated".
// git log HEAD..swiftpage/development

if(typeof $$ !== 'undefined') {
    $$.async();
    getUpdate('portal')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


reading notebook /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[3]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[4]
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[1]
reading notebook /Users/briancullinan/jupytangular2/Utilities/file system.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[5]
reading notebook /Users/briancullinan/jupytangular2/Analytics/budgeting.ipynb
reading notebook /Users/briancullin

error: pathspec 'undefined' did not match any file(s) known to git.



HEAD is now at 837e73b Merge pull request #12 from Swiftpage/Code_Model_Cleanup

creating tree "/Users/briancullinan/Documents/portal"


HEAD -> development, swiftpage/development
spawning process git branch -r
  swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal
  swiftpage/B-11309_APC_Direct_Purchase
  swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue
  swiftpage/B-11376_Account_Management_2
  swiftpage/B-11649_Convert_Product_Ids_To_Constants
  swiftpage/B-11650_Culture_From_URL
  swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model
  swiftpage/Bret_Portal_Deploy_config
  swiftpage/Code_Model_Cleanup_After_Pro_Support
  swiftpage/Only_auth
  swiftpage/UI_Cleanup
  swiftpage/development
  swiftpage/master

spawning process git fetch swiftpage
spawning process git log HEAD..swiftpage/development


Error: nothing for git to do

TODO:

Rebase squash cherrypick automatically to create perfect feature branch?

Reset each file one at a time to make sure there aren't any unnecessary changes?

Use unit testing to warn about unnecessary modules?

Load relevant projects based on branch.  It could parse the .sln file and automatically patch whenever the checked out index changes.

Plot when a TODO: item is added and when it is removed on Google Calendar, then verify by checking commit -messages, remaining comments/function names with matching keywords (split by camel casing)?, plot dates completed for measurement of self set tasks and short term goals, which activities are harder than others?  better insight in to erroneous developer time estimates?

Interface for searching all TODO: items?  Calendar?



git project directory?

get fresh git project directory?


In [ ]:
var fs = require('fs');
var glob = require('glob');
var rimraf = require('rimraf');
var path = require('path');
var importer = require('../Core');
var listInProject = importer.import('list project files');

function deepCopy(project_orig) {
    const now = new Date();
    // delete old cloned directories
    const copies = glob.sync(path.basename(project_orig) + '-copy-*', {cwd: path.dirname(project_orig)});
    copies.forEach(c => {
        rimraf.sync(path.join(path.dirname(project_orig), c));
    });
    const projectName = path.basename(project_orig) + '-copy-' + now.getFullYear()
        + '-' + (now.getMonth() + 1) + '-' + now.getDate()
        + '-' + now.getHours() + '-' + now.getMinutes()
        + '-' + now.getSeconds();
    const project = path.join(path.dirname(project_orig), projectName);
    // do a deep copy of all git files
    console.log('cloning repo ' + project_orig + ' -> ' + project);
    return Promise.all([
        listInProject(project_orig, '**/*'),
        listInProject(project_orig + '/.git', '**/*')
    ])
        // TODO: clone from remote
        .then(r => {
            for (const file of [].concat(...r).sort()) {
                const dirname = path.dirname(
                    file.replace(project_orig, project)).split('/');

                for (var i = 2; i <= dirname.length; i++) {
                    if (!fs.existsSync(dirname.slice(0, i).join('/'))) {
                        fs.mkdirSync(dirname.slice(0, i).join('/'));
                    }
                }
                fs.copyFileSync(file, file.replace(project_orig, project));
            }
        })
        .then(() => listInProject(project, '**/*.lock').forEach(p => rimraf.sync(p)))
        .then(() => project)
}

function cloneProject(project, copy) {
    var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
    if(!project) {
        throw new Error('specify project path or github');
    }
    if(fs.existsSync(path.resolve(project))) {
        project = path.resolve(project);
    } else if(fs.existsSync(path.join(PROFILE_PATH, 'Documents/' + project))) {
        project = path.resolve(path.join(PROFILE_PATH, 'Documents/' + project));
    } else if(fs.existsSync(path.join(PROFILE_PATH, project))) {
        project = path.resolve(path.join(PROFILE_PATH, project));
    } else if(project.match(/github.com\/[a-z0-9_-]+\/[a-z0-9_-]+/ig)) {
        // fetch remote
        const projectName = (/github.com\/([a-z0-9_-]+)\/([a-z0-9_-]+)/ig).exec(project)[2];
        const projectHttp = project.replace(/^(https*)*:*\/*\/*/ig, 'https://');
        if(fs.existsSync(path.join(PROFILE_PATH, projectName))) {
            project = path.resolve(path.join(PROFILE_PATH, projectName));
        } else {
            return execCmd(`
git clone -v ${projectHttp}
`, {cwd: path.resolve(PROFILE_PATH)})
                .then(r => path.resolve(path.join(PROFILE_PATH, projectName)))
                .then(r => copy ? deepCopy(r) : r)
        }
    } else {
        throw new Error('invalid project path "' + project + '"');
    }
    return Promise.resolve(project)
            .then(r => copy ? deepCopy(r) : r)
}
module.exports = cloneProject;



megamind?

github commit html acl?

TODO: parse html document and copy changes to commit


In [6]:
var GitHubApi = require('github');
var ASTQ = require('astq');
var ASTY = require('asty');
var css = require('css');

// commit changes to github
function getGist(gist) {
    const github = new GitHubApi({
        host: 'api.github.com'
    });
    /*
    github.authenticate({
        type: 'basic',
        username: process.env.USERNAME,
        password: process.env.PASSWORD
    });
    */

    return github.gists.get({id: gist})
        .then(r => r.data.files);
}

function replaceSelectors(str, prefix) {
    const ast = css.parse(str);
    ast.stylesheet.rules.forEach(r => {
        if(typeof r.selectors === 'undefined') {
            return;
        }
        r.selectors.forEach((s, i) => {
            r.selectors[i] = prefix + ' ' + s;
        });
    })
    return css.stringify(ast);
}

// git 
function gitEditor(gist, file) {
    // TODO: use a Github repo as the input
    const HOST_ID = 'webtest123'
    file = file || 'index';
    return getGist(gist || 'a572d0830ae72b962e12a57adaec7c52')
        .then(files => {
            // scan using an acl list, similar to easylist?
            // TODO: accept formats:
            //    {"selector": "selector"}
            //    {"glob-url@selector": "glob-template-path@selector"}
            //    {"selector": "html-file@selector"}
            //    {"selector": "html-file@xpath"} ?
            //    {"glob-file": {"glob-url"...} || ["selector"]}
            const paths = JSON.parse(files[ file + '-acl.json' ].content);
            const content = files[ file + '.html' ].content;
            const css = replaceSelectors(files[ file + '.css' ].content, '#' + HOST_ID);
            return {
                id: HOST_ID,
                html: '<style>' + css + '</style>' + content,
                acl: paths
            };
        })
}
module.exports = gitEditor;

if(typeof $$ !== 'undefined') {
    $$.async();
    gitEditor()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


{ id: 'webtest123',
  html: '<style>@charset "UTF-8";\n\n@font-face {\n  font-family: FontAwesome;\n  src: url(../fonts/FontAwesome/fontawesome-webfont.eot?v=4.7.0);\n  src: url(../fonts/FontAwesome/fontawesome-webfont.eot?#iefix&v=4.7.0) format("embedded-opentype"), url(../fonts/FontAwesome/fontawesome-webfont.woff2?v=4.7.0) format("woff2"), url(../fonts/FontAwesome/fontawesome-webfont.woff?v=4.7.0) format("woff"), url(../fonts/FontAwesome/fontawesome-webfont.ttf?v=4.7.0) format("truetype"), url(../fonts/FontAwesome/fontawesome-webfont.svg?v=4.7.0#fontawesomeregular) format("svg");\n  font-weight: 400;\n  font-style: normal;\n}\n\n#webtest123 .fa {\n  display: inline-block;\n  font: normal normal normal 14px/1 FontAwesome;\n  font-size: inherit;\n  text-rendering: auto;\n  -webkit-font-smoothing: antialiased;\n  -moz-osx-font-smoothing: grayscale;\n}\n\n#webtest123 .fa-lg {\n  font-size: 1.33333333em;\n  line-height: .75em;\n  vertical-align: -15%;\n}\n\n#webtest123 .fa-2x {\n  font-s